In [ ]:
!pip install geocoder
!pip install lxml
!pip install beautifulsoup4
!pip install folium

In [4]:
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import numpy as np
from itertools import chain
from sklearn.cluster import AffinityPropagation
from sklearn import metrics
import geocoder # import geocoder
from bs4 import BeautifulSoup 
import folium 

In [5]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text #request to get the html as text

In [6]:
soup = BeautifulSoup(website_url,'lxml')
My_table = soup.find('table',{'class':'wikitable sortable'}) #getting the section of the code related to the table

In [7]:
Vector=[]
test2=My_table.find_all("td")

n=0
while n<len(test2):
    Vector=Vector + [test2[n]]                        #getting just the part that has the table content
    n=n+1

i=0
while i<len(Vector):
    Vector[i]=str(Vector[i])                                             
    i=i+1
t=0
Vector_G=[]
while t<len(Vector):
    Vector_G=Vector_G + [[Vector[t],Vector[t+1],Vector[t+2]]]              #grouping into three coloumns
    n=n+1
    t=t+3

for i in Vector_G:
    for t in range (0,3):
        i[t]=i[t].replace("<td>",'')
        i[t]=i[t].replace("</td>",'')                                     #eliminating extra char. not needed
        i[t]=i[t].replace("\n",'')
        i[t]=i[t].replace('<a href="/wiki/','')      
        i[t]=i[t].replace('</a>','')
        i[t]=i[t].replace('>',' ')

x=0
while x<len(Vector_G):
    Vector_G[x][1]=Vector_G[x][1].split('"',-1)[-1]
    Vector_G[x][2]=Vector_G[x][2].split('"',-1)[-1]                         #recovering the Borough and Neighborhoods        
    x=x+1

x=0
i=0
while x<len(Vector_G):
    while i<3:
        Vector_G[x][i]=Vector_G[x][i].strip()                         #deleting blank spaces before and after each value
        i=i+1
    i=0
    x=x+1   

In [8]:
Df=pd.DataFrame(Vector_G)
Df.columns = ['PostalCode', 'Borough', 'Neighborhood']              #setting the headers
Df2=Df.copy()
Df = Df[Df.Borough != 'Not assigned']                               #dropping the rows with Borough='Not assigned'
lt=Df.values.tolist()                                 #converting to list to add the borough as neigh. when not assigned
for i in lt:
    if i[2]=='Not assigned':
        i[2]=i[1]
Df=pd.DataFrame(lt)                                            #converting to DataFrame again, setting headers again
Df.columns = ['PostalCode', 'Borough', 'Neighborhood']              

In [9]:
aggregation_functions = {'PostalCode': 'first', 'Borough': 'first', 'Neighborhood': ", ".join}
Df_group = Df.groupby(Df['PostalCode']).aggregate(aggregation_functions)
Df=Df_group.reset_index(drop='true')               #combining the coloumns with same post code, reseting index afterwards

In [10]:
Df[0:10]                                        #showing the first ten rows

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [11]:
Df.shape                       #informing the shape as per requested.

(103, 3)